In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [2]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

# after you pull run `ollama run llama3.2`

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and even entire websites.
2. **Marketing Automation**: AI-powered chatbots can help automate customer service, lead generation, and personalized marketing campaigns, providing a more efficient and effective way to engage with customers.
3. **Design and Visualization**: Generative AI can be used to create 3D models, architecture designs, product designs, and even entire brand identities, reducing the time and cost associated with traditional design methods.
4. **Personalized Recommendations**: Generative AI algorithms can analyze customer behavior and preferences to provide personalized product recommendations, improving user experience and increasing sales.
5. **Data Analysis and Insights**: Generative AI can be used to analyze large datasets, i

In [5]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as product descriptions, social media posts, and articles. This can reduce the cost and time spent on creating content, while also increasing its consistency and quality.
2. **Image and Video Generation**: Generative AI can generate realistic images and videos that can be used for various purposes such as marketing materials, advertising, and entertainment. For example, companies like Netflix use generative AI to create personalized movie recommendations and promotional content.
3. **Chatbots and Virtual Assistants**: Generative AI is being used to build more sophisticated chatbots and virtual assistants that can understand natural language and provide personalized customer support.
4. **Product Design and Development**: Generative AI can be used to generate design concepts for new products, reducin

In [6]:
!ollama pull deepseek-r1:1.5b

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest 
pulling aabd4debf0c8... 100% ▕████████████████▏ 1.1 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling a85fe2a2e58e... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 
<think>
Okay, so I need to understand the concepts behind large language models (LLMs), specifically focusing on what neural networks, attention mechanisms, and transformers are. Hmm, I've heard about NLP before, but I'm not exactly sure how all these pieces fit together.

Let's start with neural networks because they seem pretty fundamental in LLMs. From what I remember, a neural network is a computational model inspired by the st

In [7]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
result = Website("https://www.reddit.com/")
print(result.title)

Reddit - The heart of the internet


In [9]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n"
    user_prompt += website.text
    return user_prompt

In [10]:
def messages_for(website): 
    prompt = user_prompt_for(website)
    if not prompt:
        raise ValueError("user_prompt_for(website) returned an empty string!")
    
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."},
        {"role": "user", "content": prompt}
    ]

In [11]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [12]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


display_summary("https://reddit.com")

This is a post from the subreddit r/RelationshipsAmI, where users share stories and ask for advice on their relationships. The user shares a story about a recent conflict with her partner, who she claims has a history of being aggressive and mocking her.

The user describes how they went to celebrate a career milestone together, but the partner's family was present and he became focused on making them laugh at her expense. She claims that he paid for the cake and then smashed it on the floor, calling it "funny." When she tried to leave or apologize, he said that his mother felt humiliated by her reaction.

The user is now wondering if they were justified in slapping him (she didn't actually slap him) and asking for advice from others. They've blocked their partner and are seeking validation on whether their behavior was reasonable under the circumstances.

Some potential issues with this story:

1. The user's behavior: While it's understandable to want to protect oneself from hurtful comments, slapping someone is a physical act that can escalate conflicts.
2. The power dynamic: The user's partner has a history of being aggressive and mocking her, which could create an imbalance in the relationship. This might have led to feelings of frustration or defensiveness in the user.
3. The timing and context: Celebrating a career milestone with someone who is prone to mockery and aggression might not be the best idea.

Some potential questions to consider:

* Was the partner's behavior truly hurtful or was it just an attempt to make his family laugh?
* Did the user's reaction (slapping) seem proportionate to the situation, or was it a dramatic overreaction?
* Is there a deeper issue in the relationship that needs to be addressed before conflicts like this arise?

The final verdict will depend on individual perspectives and values. Some might see the user's behavior as justified, while others might think it was an overreaction or even a sign of larger relationship issues.

# Tool to answer any technical questions related to software engineering using Ollama 3.2

In [18]:
question = """Please explain how do you solve the prime numbers in Java efficiently"""

messages = [
    {"role": "system", "content": "you are a programming expert who answers questions about python, java, software engineering, LLMs and Data science"},
    {"role": "user", "content": "Please give details explanations of the followig question:" + question}
]


In [19]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

Solving Prime Numbers in Java Efficiently
=====================================

Prime numbers are a fundamental concept in number theory, and they have numerous applications in various fields, such as cryptography, coding theory, and computer science. In this explanation, we'll discuss an efficient algorithm to find prime numbers in Java.

**Overview of the Problem**

Given a range of integers, we want to identify all prime numbers within that range. A prime number is a positive integer greater than 1 that has no positive divisors other than 1 and itself.

**Approach: Sieve of Eratosthenes Algorithm**

The most efficient algorithm for finding prime numbers up to a given limit is the Sieve of Eratosthenes. This algorithm works by iteratively marking as composite (not prime) the multiples of each prime number as it is encountered.

Here's why this approach is efficient:

*   It only requires a single pass through the range of integers.
*   It uses a minimum amount of memory, making it suitable for large ranges.
*   It is relatively simple to implement.

**Java Implementation**

```java
public class SieveOfEratosthenes {

    /**
     * Finds all prime numbers up to the given limit using the Sieve of Eratosthenes algorithm.
     *
     * @param limit The upper bound (inclusive) for finding prime numbers.
     * @return A list of prime numbers within the specified range.
     */
    public static List<Integer> findPrimes(int limit) {
        boolean[] primes = new boolean[limit + 1];
        primes[0] = primes[1] = false;

        // Iterate from 2 to sqrt(limit)
        for (int p = 2; p * p <= limit; p++) {
            if (primes[p]) {
                // Mark as composite all multiples of p
                for (int i = p * p; i <= limit; i += p) {
                    primes[i] = true;
                }
            }
        }

        // Return a list of prime numbers within the specified range
        List<Integer> result = new ArrayList<>();
        for (int p = 0; p < limit; p++) {
            if (!primes[p]) {
                result.add(p);
            }
        }
        return result;
    }

    public static void main(String[] args) {
        int limit = 100;
        List<Integer> primes = findPrimes(limit);
        System.out.println("Prime numbers up to " + limit + ": " + primes);
    }
}
```

**Explanation of the Code**

The code implements the Sieve of Eratosthenes algorithm as follows:

1.  We create a boolean array `primes` with size `limit + 1`, where `primes[i]` represents whether `i` is prime or not.
2.  We initialize all elements to `false`, except for `0` and `1`, which are not prime numbers by definition.
3.  We iterate from 2 to the square root of `limit`. For each prime number `p` in this range:
    *   If `p` is marked as prime (`primes[p] == true`), we mark all its multiples `i` (starting from `p * p`) as composite by setting `primes[i] = true`.
4.  Finally, we create a list of all prime numbers within the specified range by iterating through the `primes` array and adding indices with `false` values to the result list.
5.  In the `main` method, we demonstrate how to use this implementation to find prime numbers up to a given limit.

**Time Complexity**

The time complexity of the Sieve of Eratosthenes algorithm is O(n log log n), where n is the upper bound of the range. This makes it an efficient solution for finding prime numbers within large ranges.

**Space Complexity**

The space complexity of this implementation is O(n), as we need to store a boolean array with size `limit + 1` to keep track of prime numbers.